In [1]:
pip install sentence-transformers pandas scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json

In [3]:
with open("iso27002_extracted.json", "r", encoding="utf-8") as f:
    iso_data = json.load(f)

with open("cleaned_sopsv2.json", "r", encoding="utf-8") as f:
    sop_data = json.load(f)

In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
sop_text = sop_data[3]["text"]
results = []


In [20]:
for clause_id, content in iso_data.items():
    clause_name = content.get("Clause Name", "").strip()
    control = content.get("Control", "").strip()
    purpose = content.get("Purpose", "").strip()
    guidance = content.get("Guidance", "").strip()

    if not (control or purpose or guidance):
        continue

    full_text = f"{control} {purpose} {guidance}".strip()

    embeddings = model.encode([full_text, sop_text])

    score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    results.append({
        "Clause ID": clause_id,
        "Clause Name": clause_name,
        "Similarity Score": round(score, 3)
    })

In [21]:
df = pd.DataFrame(results).sort_values(by="Similarity Score", ascending=False).reset_index(drop=True)

In [22]:
df.to_csv("ISO_to_SOP_Similarity_Report_3.csv", index=False)
print(df.head(20))

   Clause ID                                        Clause Name  \
0        6.2                T erms and conditions of employment   
1        6.1                                          Screening   
2       5.36  C ompliance with policies, rules and standards...   
3       5.27      Learning fr om information security incidents   
4        6.5  R esponsibilities after termination or change ...   
5       5.35        Independent r eview of information security   
6       5.29            Information secur ity during disruption   
7        6.3  Information secur ity awareness, education and...   
8        6.4                              Disciplinary pr ocess   
9       5.34                     Pri vacy and protection of PII   
10      5.33                             Pr otection of records   
11       8.3                    Information ac cess restriction   
12      5.30             ICT r eadiness for business continuity   
13       5.1                 P olicies for information securit